In [1]:
# import libraries
import pandas as pd
import re
#from english_words import english_words_set
from wordfreq import zipf_frequency # count the usage frequency of words

In [2]:
# wordle word pool obtained from: https://gist.github.com/cfreshman

# before nyt bought wordle
guesses_org = pd.read_csv('wordle-allowed-guesses.txt', header=None)
answers_org = pd.read_csv('wordle-answers-alphabetical.txt', header=None)

# after nyt bought wordle
guesses_nyt = pd.read_csv('wordle-nyt-allowed-guesses.txt', header=None)
answers_nyt = pd.read_csv('wordle-nyt-answers-alphabetical.txt', header=None)

In [3]:
# combine the allowed guesses and answers to create the complete allowed word list
word_pool_org = guesses_org[0].to_list() + answers_org[0].to_list()
word_pool_nyt = guesses_nyt[0].to_list() + answers_nyt[0].to_list()

In [4]:
print(len(word_pool_org))
print(len(word_pool_nyt))

12972
12947


In [5]:
# count the usage frequency of words in word_list using the wordfreq library
# more information about the library: https://pypi.org/project/wordfreq/
def create_usage_freq_d(word_list):    
    usage_freq_d = {}
    for word in word_list:
        usage_freq_d[word] = zipf_frequency(word, 'en')
        
    # sort the dict by the usage frequency score in desc order
    sorted_usage_freq_d = {k: v for k, v in reversed(sorted(usage_freq_d.items(), key=lambda item: item[1]))}
    return sorted_usage_freq_d

In [6]:
# this function is not used
def create_letter_score_d(word_list):
    letters_d = {}
    for letter in ''.join(word_list):
        if letter in letters_d:
            letters_d[letter] += 1
        else:
            letters_d[letter] = 1
        
    freq_d = {key: 0 for key in word_list}
    for word in word_list:
        unique_letter_word = "".join(set(word))
        for n in unique_letter_word:
            freq_d[word] += letters_d[n]
        
    sorted_freq_d = {k: v for k, v in reversed(sorted(freq_d.items(), key=lambda item: item[1]))}

In [7]:
def get_pattern(guess, answer):
    c = 0
    pattern = ''
    while c < 5:
        if guess[c] == answer[c]:
            pattern+='g'
        elif guess[c] in answer:
            pattern+='y'
        else:
            pattern+='b'
        c+=1
    return pattern

In [8]:
# function to update the word list based on the green/yellow/black feedback after each guess
# function returns an updated and usage frequency sorted word list
def update_pool(pool, guess, pattern):
    pattern_dict = {}
    for n in range(5):
        if list(guess)[n] not in pattern_dict:
            pattern_dict[list(guess)[n]] = list(pattern)[n]
        elif list(guess)[n]+'1' in pattern_dict:
            pattern_dict[list(guess)[n]+'2'] = list(pattern)[n]
        else:
            pattern_dict[list(guess)[n]+'1'] = list(pattern)[n]
        
    b_list = []
    for k, v in pattern_dict.items():
        if v =='b':
            b_list.append(k[0])
            
    g_list = []
    for k, v in pattern_dict.items():
        if v =='g':
            g_list.append(k[0])
            
    b_list = [letter for letter in b_list if letter not in g_list]
            
    #print('Black List:')
    #print(b_list)
    #print("")
    
    if len(b_list) != 0:
        for b_letter in b_list:
            pool = [word for word in pool if b_letter not in word]
            
    #print('After Black:')
    #print(pool)
    #print("")
            
    g_pattern = ''
    for k, v in pattern_dict.items():
        if v =='g':
            g_pattern+=k[0]
        else:
            g_pattern+='.'
    p = re.compile(g_pattern)
    pool = [word for word in pool if p.match(word)]
    
    #print("After Green:")
    #print(pool)
    #print("")
    
    count = 0
    y_dict = {}
    for k, v in pattern_dict.items():
        if v =='y':
            y_dict[k] = count
        count+=1
    
    g_list = []
    for k, v in pattern_dict.items():
        if v =='g':
            g_list.append(k[0])

    remove_list = []
    for k, v in y_dict.items():
        for word in pool:
            if (word.find(k[0]) == v):
                remove_list.append(word)
    
    pool = [word for word in pool if word not in remove_list]
    
    y_list = [y[0] for y in list(y_dict.keys())]
    if len(y_list) != 0:
        for y_letter in y_list:
            pool = [word for word in pool if y_letter in word]
    
    try:
        pool.remove(guess)
    except:
        pass
        
    # select the pairs in updated word list from the word usage frequency dictionary
    freq_d = {word_key: usage_freq_d[word_key] for word_key in pool}
    sorted_freq_d = {k: v for k, v in reversed(sorted(freq_d.items(), key=lambda item: item[1]))}
    return sorted_freq_d
    #return list(sorted_freq_d.keys())

In [9]:
usage_freq_d = create_usage_freq_d(word_pool_nyt)

In [10]:
# check the word usage frequency dictionary
list(usage_freq_d.items())[:10]

[('about', 6.4),
 ('their', 6.33),
 ('there', 6.31),
 ('which', 6.3),
 ('would', 6.27),
 ('other', 6.16),
 ('first', 6.11),
 ('after', 6.11),
 ('think', 6.08),
 ('could', 6.06)]

## Single trial

In [11]:
#test how many guesses the function need to use to solve one puzzle

sample_answer = "nymph"
guess = 'roate' # set the starting word whatever you like
guess_count = 1
word_pool = word_pool_nyt

while guess != sample_answer:
    print('Guess count:', guess_count)
    print('Guess:', guess)
    print('Feedback:', get_pattern(guess, sample_answer))
    print('')
    word_pool = update_pool(word_pool, guess, get_pattern(guess, sample_answer))
    #print(word_pool)
    guess = list(update_pool(word_pool, guess, get_pattern(guess, sample_answer)).keys())[0]
    guess_count+=1
    
print('Guess count:', guess_count)
print('Guess:', guess)
print('Feedback:', get_pattern(guess, sample_answer))
print('')

Guess count: 1
Guess: roate
Feedback: bbbbb

Guess count: 2
Guess: which
Feedback: bybbg

Guess count: 3
Guess: flush
Feedback: bbbbg

Guess count: 4
Guess: nymph
Feedback: ggggg



## Helper

In [12]:
# actually use the function to solve puzzle, without the answer
new_pool = update_pool(word_pool_nyt, 'roate', 'gbbby')
new_pool

{'rules': 5.06,
 'river': 5.03,
 'reply': 4.5,
 'refer': 4.45,
 'ruled': 4.3,
 'rebel': 4.08,
 'reign': 4.07,
 'rises': 4.07,
 'rides': 4.06,
 'rider': 4.02,
 'remix': 3.99,
 'riley': 3.9,
 'risen': 3.86,
 'ruler': 3.84,
 'renew': 3.73,
 'resin': 3.61,
 'reuse': 3.37,
 'relic': 3.37,
 'reefs': 3.36,
 'reins': 3.31,
 'repel': 3.2,
 'reels': 3.12,
 'revel': 3.11,
 'reddy': 3.08,
 'revue': 3.07,
 'rupee': 3.04,
 'reeds': 3.02,
 'reeve': 2.91,
 'rerun': 2.9,
 'runes': 2.89,
 'riser': 2.88,
 'riled': 2.84,
 'reeks': 2.81,
 'redux': 2.74,
 'riven': 2.72,
 'recur': 2.65,
 'ripen': 2.64,
 'reiki': 2.59,
 'reedy': 2.56,
 'rebus': 2.53,
 'rimes': 2.4,
 'rebbe': 2.35,
 'rumen': 2.3,
 'redid': 2.29,
 'rives': 2.23,
 'rubes': 2.22,
 'recce': 2.22,
 'riles': 2.21,
 'revie': 2.16,
 'ruder': 2.13,
 'renin': 2.09,
 'rubel': 1.96,
 'ribes': 1.96,
 'riper': 1.95,
 'rices': 1.89,
 'repin': 1.87,
 'ruses': 1.84,
 'rebuy': 1.84,
 'ricer': 1.8,
 'rheum': 1.79,
 'renne': 1.79,
 'rimer': 1.72,
 'regie': 1.66,


In [13]:
new_pool_1 = update_pool(new_pool, 'rules', 'gbbgb')
new_pool_1

{'river': 5.03,
 'refer': 4.45,
 'rider': 4.02,
 'renew': 3.73,
 'riven': 2.72,
 'ripen': 2.64,
 'riper': 1.95,
 'ricer': 1.8,
 'rimer': 1.72,
 'riced': 1.43,
 'rebec': 1.29,
 'riped': 1.22,
 'rived': 1.21,
 'rekey': 1.14,
 'remen': 1.14,
 'ricey': 1.03,
 'rimed': 1.03,
 'reded': 0.0,
 'refed': 0.0,
 'rehem': 0.0,
 'reked': 0.0,
 'remex': 0.0,
 'reney': 0.0,
 'repeg': 0.0,
 'rewed': 0.0,
 'rifer': 0.0,
 'ryked': 0.0,
 'ryper': 0.0}

In [14]:
new_pool_2 = update_pool(new_pool_1, 'river', 'gbbgb')
new_pool_2

{'refer': 4.45,
 'renew': 3.73,
 'rebec': 1.29,
 'remen': 1.14,
 'rekey': 1.14,
 'ryper': 0.0,
 'ryked': 0.0,
 'rewed': 0.0,
 'repeg': 0.0,
 'reney': 0.0,
 'remex': 0.0,
 'reked': 0.0,
 'rehem': 0.0,
 'refed': 0.0,
 'reded': 0.0}

In [15]:
new_pool_3 = update_pool(new_pool_1, 'refer', 'ggbgb')
new_pool_3

{'renew': 3.73,
 'rebec': 1.29,
 'remen': 1.14,
 'rekey': 1.14,
 'rewed': 0.0,
 'repeg': 0.0,
 'reney': 0.0,
 'remex': 0.0,
 'reked': 0.0,
 'rehem': 0.0,
 'reded': 0.0}

## Test with all words

In [16]:
%%time
#test how many guesses the function need to use to solve all the puzzles

def guessing(first_guess, answer_list, fail=0):
    
    if type(answer_list) == str:
        word_pool = word_pool_nyt
        guess_count = 1
        guess = first_guess
        while guess != answer:
            word_pool = update_pool(word_pool, guess, get_pattern(guess, answer))
            guess = list(update_pool(word_pool, guess, get_pattern(guess, answer)).keys())[0]
            guess_count+=1
        return guess_count
    
    else:
        guesses_list = []
        for answer in answer_list:
            word_pool = word_pool_nyt
            guess_count = 1
            guess = first_guess
            while guess != answer:
                word_pool = update_pool(word_pool, guess, get_pattern(guess, answer))
                guess = list(update_pool(word_pool, guess, get_pattern(guess, answer)).keys())[0]
                guess_count+=1
            guesses_list.append(guess_count)
            #print(guess, answer, guess_count)
        if fail == 1:
            fail_count = sum(guess > 6 for guess in guesses_list)
            guesses_list = [guess for guess in guesses_list if guess <= 6]
            return sum(guesses_list)/len(guesses_list), len(guesses_list)/(len(guesses_list)+fail_count)
        else:
            return sum(guesses_list)/len(guesses_list)

print("Average guess used, success percentage(solve within 6 guesses)")
guessing('plate', answers_nyt[0].to_list(), 1)

Average guess used, success percentage(solve within 6 guesses)
CPU times: user 1min 5s, sys: 599 ms, total: 1min 5s
Wall time: 1min 9s


(4.026749888542131, 0.97141619748809)

### Using 'plate' as starting word, average guess used: 4.03, success percentage(solve within 6 guesses): ~97%